<span style="font-family: Arial; color:green; font-size:1.6em; font-weight:bold" >Tutorial: Aumentando o Poder Preditivo de Seus Modelos de Machine Learning com Stacking Ensembles</span>

Tutorial for improve skills: 'Tutorial: Aumentando o Poder Preditivo de Seus Modelos de Machine Learning com Stacking Ensembles' (from ledmaster - Mario Filho) by Marcus Mariano

**For more information about Marcus Mariano: [Web site](https://marcusmariano.github.io/mmariano/)**  

**utorial: Aumentando o Poder Preditivo de Seus Modelos de Machine Learning com Stacking Ensembles [here](https://github.com/ledmaster/TutorialEnsemble/blob/master/HomePrices.ipynb)** 

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Ensembles - são conjuntos de modelos que oferecem uma performance melhor do que cada modelo que o compõe.

Então neste artigo fala sobre melhor maneira que para criar __ensembles: stacking__. 

In [2]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

sns.set(style="darkgrid", color_codes=True)
%matplotlib inline

## Carregando os dados

Os dados utilizados são transações comerciais de imóveis na cidade de Ames, Iowa. Nosso objetivo é prever o preço de venda de uma casa alimentando o modelo com as características. 

Estes dados também são tema de uma competição no Kaggle. Os dados de treino e teste podem ser encontrados em: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/

No total temos 79 colunas, mas limitei a 15 aqui por causa da formatação do site.

In [3]:
base = pd.read_csv('data/train.csv', index_col='Id')
X, y = base.drop('SalePrice', axis=1), base.SalePrice.copy()
print(base.shape)
base.head().iloc[:, :13]

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1
Id,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm


In [4]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [42]:
# base['SalePrice'].unique()
len(base['SalePrice'].unique())

663

In [4]:
# train.info()
base.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

# Conjuntos de Variáveis (Features)

Uma das maneiras de obter modelos diferentes é variar a representação dos dados utilizada para treiná-los. Por isso nosso primeiro passo será construir estas representações.

__Funções auxiliares__

A métrica sugerida pelo Kaggle para avaliar os modelos é o RMSLE, este erro leva em conta a diferença entre o logaritmo das previsões e do alvo. É possível pensar neste erro como uma aproximação do erro percentual do modelo, mas com propriedades mais interessantes do ponto de vista matemático.

Para usarmos algumas funções do Scikit-learn que vão ajudar a manter nosso código mais limpo e mais eficiente, precisamos criar as __funções de erro da maneira requerida pelo módulo__. Neste caso, a função pecisa receber um modelo treinado, as features e o alvo. Ela computará as previsões e deverá retornar um número, que é o valor da métrica de erro.

A última linha da célula cria um objeto da validação cruzada do scikit-learn. Ele vai cuidar da divisão dos dados para nós. 



## Funções de Erro

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor

def rmsle(model, X, y):
    p = model.predict(X)
    return np.sqrt(mean_squared_error(y, p))

def rmsle_log_y(model, X, y):
    p = model.predict(X)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))    

def rmsle_sqrt_y(model, X, y):
    p = model.predict(X)
    y = np.power(y, 2)
    p = np.power(p, 2)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))



In [6]:

kf = KFold(n_splits=5, shuffle=True, random_state=1)



# RandomForestRegressor

### Feature set 1: variáveis "numéricas"

O primeiro conjunto de features que teremos será uma seleção simples das variáveis originalmente numéricas dos dados. Para isto basta selecionar todas as colunas que possuem variáveis com números inteiros ou de ponto flutuante. Além disso, para substituir os valores nulos (o scikit-learn exige a substituição), decidi colocar o valor -1. 

Após armazenar estes dados na variável X1, vemos que são 36 colunas. Já nesta parte quero treinar um modelo de Random Forest dentro da validação cruzada, para sabermos como ele se sairia sozinho nos dados originais.

In [7]:
SEED = 0 

### X1 - Numeric values

In [8]:
X1 = X.select_dtypes(include=[np.number]).fillna(-1)

print('Dims', X1.shape)

rfr = RandomForestRegressor(n_estimators=1000, 
                              random_state=SEED)

Dims (1460, 36)


In [9]:
X1.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


### RandomForestRegressor X1 y

In [10]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

error = cross_val_score(rfr, 
                        X1, 
                        y, 
                        cv = kf, 
                        scoring = rmsle, 
                        verbose = 1).mean()

print('RMSLE:', error)

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSLE: 29677.20887342271


Time Elapsed: 0:01:47.164299


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.8min finished


### Feature set 2: Ordinal Encoding Categóricas

Agora vamos criar um outro conjunto de features, desta vez adicionando as variáveis categóricas. Existem várias maneiras de codificar este tipo de variável para os modelos, uma delas é usando um formato ordinal. Isso simplesmente significa substituir cada valor original por números sequenciais. Em alguns modelos isso pode ser problemático, pois eles tentarão capturar alguma relação de ordem em valores que podem não ter. No nosso caso, com modelos baseados em árvores de decisão, este problema é quase inexistente.

Após codificar desta maneira, rodamos novamente a validação cruzada, agora nestes novos dados.

### X2 - Numeric and Ordinal Encoding

In [11]:
from sklearn.preprocessing import LabelEncoder


X2 = X.copy()
for col in X2.columns:
    if X2[col].dtype == object:
        enc = LabelEncoder()
        X2[col] = enc.fit_transform(X[col].fillna('Missing'))

print('Dims', X2.shape)
X2.fillna(-1, inplace=True)

rfr = RandomForestRegressor(n_estimators=1000, 
                              random_state=SEED)


Dims (1460, 79)


In [14]:
X2.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,3,65.0,8450,1,1,3,3,0,4,...,0,0,3,2,1,0,2,2008,8,4
2,20,3,80.0,9600,1,1,3,3,0,2,...,0,0,3,2,1,0,5,2007,8,4
3,60,3,68.0,11250,1,1,0,3,0,4,...,0,0,3,2,1,0,9,2008,8,4
4,70,3,60.0,9550,1,1,0,3,0,0,...,0,0,3,2,1,0,2,2006,8,0
5,60,3,84.0,14260,1,1,0,3,0,2,...,0,0,3,2,1,0,12,2008,8,4


### RandomForestRegressor X2

In [12]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

error = cross_val_score(rfr, 
                        X2, 
                        y, 
                        cv = kf, 
                        scoring = rmsle, 
                        verbose = 1).mean()

print('RMSLE:', error)

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSLE: 29211.70944880934


Time Elapsed: 0:02:18.618152


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.3min finished


### Bônus: OneHot Encoding Categóricas

A maneira mais popular de codificar variáveis categóricas é o One Hot Encoding. Basicamente consiste em transformar cada valor da variável em uma coluna cujo novo valor será 1 caso a variável tenha aquele valor num determinado exemplo, ou 0 em caso negativo. 

Este método cria mais de 200 novas colunas, o que deixa o processo de treino mais devagar, então decidi deixar a linha da validação cruzada comentada. Caso queira ver o resultado, basta rodá-la sem o jogo da velha.

### X3 - Numeric values and OneHot Encoding

In [13]:
#from sklearn.preprocessing import OneHotEncoder
X3 = X.copy()
cats = []
for col in X3.columns:
    if X3[col].dtype == object:
        X3 = X3.join(pd.get_dummies(X3[col], prefix=col), how='left')
        X3.drop(col, axis=1, inplace=True)
    

print('Dims', X3.shape)
X3.fillna(-1, inplace=True)


rfr = RandomForestRegressor(n_estimators=1000, 
                              random_state=SEED)

Dims (1460, 288)


In [16]:
X3.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


### RandomForestRegressor X2 y

In [14]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

error = cross_val_score(rfr, 
                        X3, 
                        y, 
                        cv = kf, 
                        scoring = rmsle, 
                        verbose = 1).mean()

print('RMSLE:', error)

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSLE: 29272.96102308132


Time Elapsed: 0:03:07.412986


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.1min finished


## Transformações do Target

Uma maneira interessante de criar diversidade, e às vezes até obter uma melhor performance, num caso de regressão, é transformar a variável que estamos tentando prever. Neste caso testaremos duas transformações: logaritmo e raiz quadrada.

### Log

É possível ver que tentar prever o logaritmo do preço nos dá um resultado melhor. Isto acontece não só pelo fato do modelo capturar padrões diferentes, mas também porque usamos uma métrica baseada na diferença de logaritmos.

### RandomForestRegressor X1 log y

In [16]:
rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED)


error = cross_val_score(rfr, 
                        X1, 
                        np.log1p(y), 
                        cv=kf, 
                        scoring = rmsle_log_y, 
                        verbose = 1).mean()

print('RF, X1, log-target RMSLE:', error)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RF, X1, log-target RMSLE: 0.011246499475042627


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min finished


### RandomForestRegressor X2 log y

In [17]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED)
error = cross_val_score(rfr, 
                        X2, 
                        np.log1p(y), 
                        cv=kf, 
                        scoring = rmsle_log_y, 
                        verbose = 1).mean()
print('RF, X2, log-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RF, X2, log-target RMSLE: 0.011021826961724839


Time Elapsed: 0:01:58.798439


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.0min finished


### RandomForestRegressor X3 log y

In [19]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED)
error = cross_val_score(rfr, 
                        X3, 
                        np.log1p(y), 
                        cv=kf, 
                        scoring = rmsle_log_y, 
                        verbose = 1).mean()

print('RF, X3, log-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RF, X2, log-target RMSLE: 0.011061245248234531


Time Elapsed: 0:03:33.765811


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.6min finished


### Raiz Quadrada

Esta transformação também nos dá um resultado melhor do que usar a variável em seu estado original. Uma das sugestões da razão pela qual vemos este efeito é que estas transformações fazem com que a variável y tenha uma distribuição mais próxima da normal, o que facilita o trabalho do modelo.

### RandomForestRegressor X1 sqrt y

In [20]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED)

error = cross_val_score(rfr, 
                        X1, 
                        np.sqrt(y), 
                        cv=kf, 
                        scoring = rmsle_sqrt_y, 
                        verbose = 1).mean()

print('RF, X1, sqrt-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RF, X1, sqrt-target RMSLE: 0.14587396026910815

Time Elapsed: 0:01:49.142160


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.8min finished


### RandomForestRegressor X2 sqrt y

In [21]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED)

error = cross_val_score(rfr, 
                        X2, 
                        np.sqrt(y), 
                        cv=kf, 
                        scoring = rmsle_sqrt_y, 
                        verbose = 1).mean()

print('RF, X2, sqrt-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RF, X2, sqrt-target RMSLE: 0.14297527750581596

Time Elapsed: 0:02:42.045742


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.7min finished


### RandomForestRegressor X3 sqrt y

In [22]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED)

error = cross_val_score(rfr, 
                        X3, 
                        np.sqrt(y), 
                        cv=kf, 
                        scoring = rmsle_sqrt_y, 
                        verbose = 1).mean()

print('RF, X3, sqrt-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RF, X3, sqrt-target RMSLE: 0.1435792363415055

Time Elapsed: 0:03:32.142307


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.5min finished


# GradientBoostingRegressor

## Gerando modelos com modelos/algoritmos diferentes

Outra maneira de gerar diversidade para o ensemble é gerar modelos diferentes. Neste caso vou usar meu modelo preferido, o GBM. Este também é baseado em árvores de decisão, mas basicamente treina cada árvore sequencialmente focando nos erros cometidos pelas anteriores.

Nas células abaixo é possível ver a performance deste modelo nos feature sets e transformações que usamos com a Random Forest. Vemos que ele traz uma melhora significativa, capturando melhor os padrões da relação entre as variáveis e o preço de venda dos imóveis.

### GradientBoostingRegressor X1 y

In [24]:
from sklearn.ensemble import GradientBoostingRegressor


from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


gbr = GradientBoostingRegressor(random_state=SEED)


error = cross_val_score(gbr, 
                        X1, 
                        y, 
                        cv=kf, 
                        scoring = rmsle, 
                        verbose = 1).mean()

print('GBM, X1, log-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X1, log-target RMSLE: 27359.625155747468

Time Elapsed: 0:00:05.165593


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.1s finished


### GradientBoostingRegressor X1 log y

In [25]:
from sklearn.ensemble import GradientBoostingRegressor


from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


gbr = GradientBoostingRegressor(random_state=SEED)


error = cross_val_score(gbr, 
                        X1, 
                        np.log1p(y), 
                        cv=kf, 
                        scoring=rmsle_log_y, 
                        verbose = 1).mean()

print('GBM, X1, log-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X1, log-target RMSLE: 0.010348438651993867

Time Elapsed: 0:00:04.523422


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.4s finished


### GradientBoostingRegressor X2 log y

In [26]:
from sklearn.ensemble import GradientBoostingRegressor

from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

gbr = GradientBoostingRegressor(random_state=SEED)

error = cross_val_score(gbr, 
                        X2,
                        np.log1p(y), 
                        cv=kf, 
                        scoring = rmsle_log_y, 
                        verbose = 1).mean()

print('GBM, X2, log-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X2, log-target RMSLE: 0.010041427435511108

Time Elapsed: 0:00:06.109587


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.0s finished


### GradientBoostingRegressor X3 log y

In [27]:
from sklearn.ensemble import GradientBoostingRegressor

from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

gbr = GradientBoostingRegressor(random_state=SEED)

error = cross_val_score(gbr, 
                        X3,
                        np.log1p(y), 
                        cv=kf, 
                        scoring = rmsle_log_y, 
                        verbose = 1).mean()

print('GBM, X3, log-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X3, log-target RMSLE: 0.009928449479231539

Time Elapsed: 0:00:10.771306


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.7s finished


## SQRT

### GradientBoostingRegressor X1 y

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
                
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

gbr = GradientBoostingRegressor(random_state=SEED)

error = cross_val_score(gbr, 
                        X1, 
                        y, 
                        cv=kf, 
                        scoring=rmsle, 
                        verbose = 1).mean()

print('GBM, X1, target RMSLE:', error)

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X1, target RMSLE: 27359.625155747468


Time Elapsed: 0:00:03.689269


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.6s finished


### GradientBoostingRegressor X1 sqrt y

In [30]:
from sklearn.ensemble import GradientBoostingRegressor
                
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

gbr = GradientBoostingRegressor(random_state=SEED)

error = cross_val_score(gbr, 
                        X1, 
                        np.sqrt(y), 
                        cv=kf, 
                        scoring=rmsle_sqrt_y, 
                        verbose = 1).mean()

print('GBM, X1, sqrt-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X1, sqrt-target RMSLE: 0.13466955860674396

Time Elapsed: 0:00:03.838672


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.7s finished


### GradientBoostingRegressor X2 sqrt y

In [31]:
from sklearn.ensemble import GradientBoostingRegressor

from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

gbr = GradientBoostingRegressor(random_state=SEED)

error = cross_val_score(gbr, 
                        X2, 
                        np.sqrt(y), 
                        cv=kf, 
                        scoring = rmsle_sqrt_y, 
                        verbose = 1).mean()

print('GBM, X2, sqrt-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X2, sqrt-target RMSLE: 0.13107304131368688

Time Elapsed: 0:00:04.860276


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.8s finished


### GradientBoostingRegressor X3 sqrt y

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

gbr = GradientBoostingRegressor(random_state=SEED)

error = cross_val_score(gbr, 
                        X3, 
                        np.sqrt(y), 
                        cv=kf, 
                        scoring = rmsle_sqrt_y, 
                        verbose = 1).mean()

print('GBM, X3, sqrt-target RMSLE:', error)

# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GBM, X3, sqrt-target RMSLE: 0.1300452096854659

Time Elapsed: 0:00:06.627933


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.5s finished


## Stacking

Tudo o que fizemos acima é para que possamos criar o nosso ensemble. Esta é a hora juntarmos os métodos usados para melhorarmos o poder preditivo dos nossos modelos.

O __Stacking__ é uma maneira de fazer o ensemble na qual usamos modelos para fazer previsões, e depois usamos estas previsões como features em novos modelos, no que pode ser chamado de "segundo nível". Você pode fazer este processo várias vezes, mas a cada nível o retorno em performance com relação à computação necessária é menor.

No nosso caso específico, criaremos previsões usando todas as combinações de modelos (RF e GBM), transformações do target (log e raiz quadrada) e feature sets (X1 e X2). 

No fim teremos previsões de 8 modelos no primeiro nível. No segundo nível usei uma regressão linear regularizada (Ridge). Tendo estas previsões podemos computar o erro nos dados fora das nossas amostras de treino e validação internas, o que nos dará uma estimativa confiável do erro do ensemble.

In [21]:
from itertools import product
from sklearn.linear_model import Ridge

kf_out = KFold(n_splits=5, 
               shuffle=True, 
               random_state=1)

kf_in = KFold(n_splits=5, 
              shuffle=True, 
              random_state=2)

from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...

cv_mean = []

for fold, (tr, ts) in enumerate(kf_out.split(X, y)):
    
    X1_train, X1_valid = X1.iloc[tr], X1.iloc[ts]
    X2_train, X2_valid = X2.iloc[tr], X2.iloc[ts]
    
    y_train, y_valid = y.iloc[tr], y.iloc[ts]
    
    modelos = [GradientBoostingRegressor(random_state=SEED), 
               RandomForestRegressor(random_state=SEED)]
    
    targets = [np.log1p, np.sqrt]    
    feature_sets = [(X1_train, X1_valid), (X2_train, X2_valid)]
    
    
    preds_cv = []
    preds_valid = []
    
    for model, target, feature_set in product(modelos, targets, feature_sets):
        preds_cv.append(cross_val_predict(model, 
                                        feature_set[0], 
                                        target(y_train), 
                                        cv=kf_in).reshape(-1, 1))
        
        model.fit(feature_set[0], target(y_train))
        ptest = model.predict(feature_set[1])
        preds_valid.append(ptest.reshape(-1,1))
    
    preds_cv = np.concatenate(preds_cv, axis=1)
    preds_valid = np.concatenate(preds_valid, axis=1)
    
    stacker = Ridge()
    stacker.fit(preds_cv, np.log1p(y_train))
    
    error = rmsle_log_y(stacker, preds_valid, np.log1p(y_valid))
    
    cv_mean.append(error)
    
    print(f"RMSLE Fold {fold} - RMSLE {error:.4f}")


print(f"\nRMSLE CV5 {np.mean(cv_mean):4f}") 


# End Measure time elapsed
end = timer()
print(f'\nTime Elapsed: {timedelta(seconds = end - start)}')  


RMSLE Fold 0 - RMSLE 0.0099
RMSLE Fold 1 - RMSLE 0.0113
RMSLE Fold 2 - RMSLE 0.0097
RMSLE Fold 3 - RMSLE 0.0104
RMSLE Fold 4 - RMSLE 0.0084

RMSLE CV5 0.009953

Time Elapsed: 0:05:25.111663


Como podemos ver, nosso melhor modelo de primeiro nível é o GBM treinado com a transformação log nos dados X2, que atinge o erro de 0,1298. Nosso ensemble atinge o valor de 0,1290. Uma melhora de 0,62%. 

O objetivo deste artigo era demonstrar o método, sem se preocupar muito com a performance no fim. Um ensemble feito visando melhora de performance pode apresentar um resultado mais significativo.

Em alguns casos, como em fundos de investimento ou aplicações de saúde, uma melhora pequena pode ter um resultado bastante significativo no mundo real, que justifique a criação de uma solução mais completa usando stacking.

Como sempre na aplicação de Machine Learning, nada é garantia de sucesso, mas este método é um dos mais consistentes em oferecer uma melhora.